### Topic modeling/ comparing with tf-idf

In [1]:
import pandas as pd
import nltk
import re
import requests
import json
import io
import matplotlib.pyplot as plt
import numpy as np
import os.path
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
noscemus_metadata = pd.read_csv('res_df.csv')
noscemus_metadata.head(5)

,Unnamed: 0,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Original,...,Chemistry,NaN,Biology,Astronomy,Cartography,Cosmography,Medicine,Alchemy,Astrology,Earth-sciences
0,0,"Achrelius, Daniel",Scientiarum magnes recitatus publice anno 1690...,NaN,1690,[Turku],Wall,17th century,Oration,Scientiarum magnes(Google Books),...,True,False,True,True,False,True,True,True,True,False
1,1,"Acidalius, Valens","Ad Iordanum Brunum Nolanum, Italum","Poematum Iani Lernutii, Iani Gulielmi, Valenti...",1603,Liegnitz,"Albert, David",17th century,Panegyric poem,Ad Iordanum Brunum (1603)(CAMENA)Ad Iordanum B...,...,False,False,False,True,False,True,False,False,True,False
2,3,"Acidalius, Valens","Ad Iordanum Brunum Nolanum, Italum","Poematum Iani Lernutii, Iani Gulielmi, Valenti...",1603,Wrocław,"Albert, David",17th century,Panegyric poem,Ad Iordanum Brunum (1603)(CAMENA)Ad Iordanum B...,...,False,False,False,True,False,True,False,False,True,False
3,5,"Acosta, José de",De natura novi orbis libri duo et De promulgat...,NaN,1589,Salamanca,Guillelum Foquel,16th century,Monograph,De natura novi orbis(Biodiversity Heritage Lib...,...,False,False,False,True,True,True,False,False,True,False
4,6,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",NaN,1620,Heidelberg,"Rosa, Geyder",17th century,Biography,Vitae Germanorum medicorum(MDZ)Alternative lin...,...,False,False,False,False,False,False,True,False,False,False


In [3]:
def get_json_content(disc_name):
    res_txt = []
    for index, row in noscemus_metadata.iterrows():
        if row[disc_name] == True:
            id = row['id']
            base_url = "https://ccs-lab.zcu.cz/noscemus_sents_data/{}.json"
            try:
                res = requests.get(base_url.format(str(id))).json()
                #json.load((requests.get(base_url.format(str(id))).content))
                res_txt.append(res)
            except:
                print('No file')
    return res_txt


In [ ]:
latin_stopwords = {
    'et', 'in', 'de', 'non', 'cum', 'ad', 'per', 'ex', 'pro', 'ab',
    'si', 'aut', 'sed', 'ut', 'quam', 'nam', 'nec', 'quoque', 'quod',
    'enim', 'vel', 'ac', 'atque', 'ne', 'dum', 'nunc', 'tamen'
}

def get_top_words(texts, n=10):
    # Vectorize with TF-IDF
    vectorizer = TfidfVectorizer(stop_words=list(latin_stopwords), encoding='utf-8')
    tfidf_matrix = vectorizer.fit_transform(texts)
    
    # Sum scores and extract keywords
    scores = np.asarray(tfidf_matrix.sum(axis=0)).flatten()
    features = vectorizer.get_feature_names_out()
    word_scores = sorted(zip(features, scores), key=lambda x: x[1], reverse=True)
    
    return word_scores[:n]

: 

In [ ]:
# Get top keywords
text_bio = get_json_content('Biology')
#print(text_bio)
bio_keywords = get_top_words(text_bio, n=10)
text_med = get_json_content('Medicine')
med_keywords = get_top_words(text_med, n=10)

# Print results
print("Top Biology Keywords:")
for word, score in bio_keywords:
    print(f"{word}: {score:.4f}")

print("Top Medicine Keywords:")
for word, score in med_keywords:
    print(f"{word}: {score:.4f}")